In [6]:
import numpy as np
import tensorflow as tf
import json
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding
from CocktailEmbeddingMaker import CocktailEmbeddingMaker

class RecipeGenerationModel:
    def __init__(self, cocktail_embedding_maker, max_recipe_length=10):
        self.cocktail_embedding_maker = cocktail_embedding_maker
        self.ingredient_ids = cocktail_embedding_maker.ingredient_ids
        self.num_ingredients = cocktail_embedding_maker.num_ingredients
        self.max_recipe_length = max_recipe_length
        self.ingredient_embedding_matrix = cocktail_embedding_maker.create_ingredient_embedding_matrix()
        self.model = self.build_model()

    def build_model(self):
        model = Sequential([
            Embedding(self.num_ingredients, self.ingredient_embedding_matrix.shape[1],
                      weights=[self.ingredient_embedding_matrix], input_length=self.max_recipe_length, trainable=False),
            LSTM(128, return_sequences=True),
            LSTM(128),
            Dense(64, activation='gelu'),
            Dense(self.num_ingredients, activation='softmax')
        ])
        model.compile(loss='categorical_crossentropy', optimizer='adam')
        return model

    def train(self, recipes, epochs=50, batch_size=32):
        ingredient_sequences = []
        next_ingredients = []

        for recipe in recipes:
            sequence = [self.ingredient_ids[self.cocktail_embedding_maker.normalize_string(ingredient)] for ingredient in recipe]
            for i in range(1, len(sequence)):
                ingredient_sequences.append(sequence[:i])
                next_ingredients.append(sequence[i])

        ingredient_sequences = tf.keras.preprocessing.sequence.pad_sequences(ingredient_sequences, maxlen=self.max_recipe_length)
        next_ingredients = tf.keras.utils.to_categorical(next_ingredients, num_classes=self.num_ingredients)

        self.model.fit(ingredient_sequences, next_ingredients, epochs=epochs, batch_size=batch_size)

    def generate_recipe(self, seed_ingredient, user_preference, max_length=10):
        generated_recipe = [seed_ingredient]

        for _ in range(max_length - 1):
            sequence = [self.ingredient_ids[self.cocktail_embedding_maker.normalize_string(ingredient)] for ingredient in generated_recipe]
            sequence = tf.keras.preprocessing.sequence.pad_sequences([sequence], maxlen=self.max_recipe_length)

            probabilities = self.model.predict(sequence)[0]
            probabilities[sequence[0]] = 0  # 중복 재료 제거

            # 사용자 선호도를 반영하여 재료 선택 확률 조정
            for ingredient_id, prob in enumerate(probabilities):
                ingredient_name = list(self.ingredient_ids.keys())[list(self.ingredient_ids.values()).index(ingredient_id)]
                ingredient_taste_score = self.get_ingredient_taste_score(ingredient_name, user_preference)
                probabilities[ingredient_id] *= ingredient_taste_score
                    # 도수 제한 조건 추가
                ingredient_info = next((item for item in self.cocktail_embedding_maker.flavor_data if item["name"] == ingredient_name), None)
                if ingredient_info and (ingredient_info['ABV'] < user_preference['abv_min'] or ingredient_info['ABV'] > user_preference['abv_max']):
                    probabilities[ingredient_id] = 0
            next_ingredient_id = np.argmax(probabilities)
            next_ingredient = list(self.ingredient_ids.keys())[list(self.ingredient_ids.values()).index(next_ingredient_id)]
            generated_recipe.append(next_ingredient)
        # 양추정하는 부분이 들어가야함
        # 레시피 후처리: 총 도수 확인 및 재료 조정
        total_abv = self.calculate_recipe_abv(generated_recipe)
        if total_abv < user_preference['abv_min'] or total_abv > user_preference['abv_max']:
            # 도수에 맞게 재료를 조정하거나 레시피를 재생성하는 로직
            
            pass
        return generated_recipe

    def get_ingredient_taste_score(self, ingredient_name, user_preference):
        #ABV를 제외한 맛 특성 점수를 0~1사이로 정규화 함
        ingredient_info = next((item for item in self.cocktail_embedding_maker.flavor_data if item["name"] == ingredient_name), None)

        if ingredient_info:
            taste_scores = {taste: ingredient_info[taste] / 100 for taste in user_preference if (taste != 'abv' and taste != 'abv_min' and taste != 'abv_max')}
            abv_score = ingredient_info['ABV'] * user_preference['abv']
            taste_score = sum(taste_scores[taste] * user_preference[taste] for taste in taste_scores)
            return taste_score + abv_score
        else:
            return 1.0
        
    def calculate_recipe_abv(self, recipe):
        total_abv = 0
        for ingredient in recipe:
            ingredient_info = next((item for item in self.cocktail_embedding_maker.flavor_data if item["name"] == ingredient), None)
            if ingredient_info:
                total_abv += ingredient_info['ABV']
        return total_abv / len(recipe)


# 데이터 로드
with open('./train_data.json', 'r') as f:
    json_data = json.load(f)
with open('../flavor.json', 'r') as f:
    flavor_data = json.load(f)

# CocktailEmbeddingMaker 인스턴스 생성
cocktail_embedding_maker = CocktailEmbeddingMaker(json_data, flavor_data)

# RecipeGenerationModel 인스턴스 생성
recipe_generation_model = RecipeGenerationModel(cocktail_embedding_maker, max_recipe_length=10)

# 학습 데이터 준비
train_recipes = [recipe['recipe'].keys() for recipe in json_data['cocktail_info']]

# 모델 학습
recipe_generation_model.train(train_recipes, epochs=50, batch_size=32)


Epoch 1/50


c:\Users\aimed-lab-1\anaconda3\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


50/50 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - loss: 5.5013
Epoch 2/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 4.8862
Epoch 3/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 4.7354
Epoch 4/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 4.5784
Epoch 5/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 4.4259
Epoch 6/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 4.2377
Epoch 7/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 4.1355
Epoch 8/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 3.9823
Epoch 9/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 3.8889
Epoch 10/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 3.7021
Epoch 11/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 3.5536
Epoch 12/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 3.3812
Epoch 13/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 3.2696
Epoch 14/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 3.2430
Epoch 15/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 3.0642
Epoch 16/50
50/50 ━━━━━━━━━━━

In [7]:

# 사용자 선호도 설정
user_preference = {
    'sweet': 80,
    'sour': 60,
    'bitter': 20,
    'fruity': 90,
    'abv': 30,
    'abv_min': 20,
    'abv_max': 40
}

# 새로운 레시피 생성
seed_ingredient = 'orange juice'
generated_recipe = recipe_generation_model.generate_recipe(seed_ingredient, user_preference, max_length=5)

print(f"Generated Recipe: {generated_recipe}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 191ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
Generated Recipe: ['orange juice', 'rum', 'vodka', 'blue curacao', 'triple sec']
